**NASA-Inspired Research Assistant & Retrocompatibility Validation**

Copyright 2025, Denis Rothman

**Goal:** This notebook serves a dual purpose. Its primary goal is to demonstrate our fully upgraded Chapter 7 Context Engine as a complete, functional application: the **NASA-Inspired Research Assistant**. We will also perform rigorous validation by running **backward compatibility tests** for the key workflows from Chapters 5 and 6. This process showcases not only the engine's new capabilities for verifiability and security but also its stability as an evolving, enterprise-grade system.

This notebook demonstrates the following key capabilities:

* **System Validation & Backward Compatibility:** Before testing the new features, we will run the key workflows from previous chapters to prove that our latest upgrades have not compromised existing functionality, demonstrating the stability of our modular architecture.

* **High-Fidelity RAG in Action:** See the upgraded `Researcher` agent retrieve information from our new knowledge base and programmatically generate citations for its claims.

* **Agent Defenses at Work:** Observe how the `helper_sanitize_input` function provides a silent but essential layer of security, protecting the engine from potentially tainted data.

* **Complex, Dynamic Planning:** Analyze the technical trace to deconstruct the sophisticated, multi-step plan the `Planner` autonomously creates to answer a multi-faceted research question.

* **Verifiable, End-to-End Output:** The final result will be a clear, synthesized answer followed by a list of the source documents used, proving the system is not just powerful, but trustworthy.

# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch7/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch7/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch7/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch7/registry.py.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch7/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent

In [2]:
# This cell will be deleted when repo goes public
import glob

# glob.glob("/content/*.py") will return a list of matching files.
# If the list is not empty, it evaluates to True.
# If the list is empty (no .py files found), it evaluates to False.

if glob.glob("/content/*.py"):
    filesdownloaded = True
else:
    filesdownloaded = False

print(f"Are .py files present in /content? {filesdownloaded}")

import os
import glob
from google.colab import userdata

# --- Configuration ---
OWNER = "Denis2054"
REPO = "Context-Engineering"
BRANCH = "main"

FILES_TO_DOWNLOAD = [
    {"remote": "commons/ch7/utils.py", "local": "utils.py"},
    {"remote": "commons/ch7/helpers.py", "local": "helpers.py"},
    {"remote": "commons/ch7/agents.py", "local": "agents.py"},
    {"remote": "commons/ch7/registry.py", "local": "registry.py"},
    {"remote": "commons/ch7/engine.py", "local": "engine.py"}
]
# --- End Configuration ---
# Check if all target files already exist
all_files_exist = all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD)

if not all_files_exist:
    print("ℹ️ Files not found. Attempting download using private token...")

    try:
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            print("🛑 Error: 'GITHUB_TOKEN' not found in Colab Secrets.")
        else:
            # Loop and download each file that is missing
            for f in FILES_TO_DOWNLOAD:
                if not os.path.exists(f["local"]):
                    # --- FIX: Corrected the typo 'httpsfs://' to 'https://' ---
                    api_url = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{f['remote']}?ref={BRANCH}"

                    # This is the !curl command, run from Python
                    # We use os.system to execute it
                    print(f"Downloading {f['local']}...")
                    command = (
                        f"curl -Lfs "
                        f"-H \"Authorization: Bearer {token}\" "
                        f"-H \"Accept: application/vnd.github.v3.raw\" "
                        f"\"{api_url}\" -o \"{f['local']}\""
                    )

                    # os.system returns 0 on success
                    if os.system(command) != 0:
                        print(f"🛑 Error downloading {f['local']}.")

            # Final check
            if all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD):
                print("✅ All files successfully downloaded.")
            else:
                print("🛑 Some files failed to download.")

    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("✅ Files already present. Skipping download.")



Are .py files present in /content? False
ℹ️ Files not found. Attempting download using private token...
✅ All files successfully downloaded.


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title  Chapter 7: CONTROL DECK: NASA Research Assistant

# 1. Define a research goal that requires verifiable, cited answers.
goal = "What are the primary scientific objectives of the Juno mission, and what makes its design unique? Please cite your sources."

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc)

In [ ]:
#@title Chapter 6 CONTROL DECK: Context Reduction Workflow

# 1. Define a large piece of text that would be expensive or too long
# to use as direct context for the Writer agent.
large_text_from_researcher = """
Juno is a NASA space probe orbiting the planet Jupiter. It was launched from Cape Canaveral Air Force Station on August 5, 2011, as part of the New Frontiers program. Juno entered a polar orbit of Jupiter on July 5, 2016, to begin a scientific investigation of the planet. After completing its primary mission, it received a mission extension. Juno's mission is to measure Jupiter's composition, gravitational field, magnetic field, and polar magnetosphere. It is also searching for clues about how the planet formed, including whether it has a rocky core, the amount of water present within the deep atmosphere, mass distribution, and its deep winds, which can reach speeds up to 618 kilometers per hour (384 mph). Juno is the second spacecraft to orbit Jupiter, after the nuclear-powered Galileo orbiter, which orbited from 1995 to 2003. Unlike all earlier spacecraft to the outer planets, Juno is powered by solar arrays, which are commonly used by satellites orbiting Earth and working in the inner Solar System, whereas radioisotope thermoelectric generators are commonly used for missions to the outer Solar System and beyond. For Juno, however, the three largest solar array wings ever deployed on a planetary probe play an integral role in stabilizing the spacecraft and generating power.
"""

# 2. Define a goal that requires both using the large text AND a creative step.
# This forces the Planner to recognize the need for summarization before writing.
goal = f"""First, summarize the following text about the Juno probe to extract only the key facts about its scientific mission and instruments. Then, using that summary, write a short, suspenseful scene for a children's story about the probe's dangerous arrival at Jupiter.

--- TEXT TO USE ---
{large_text_from_researcher}
"""

# 3. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 4. Call the execution function
execute_and_display(goal, config, client, pc)

In [ ]:
#@title Chapter 5 Writing a story
# Define the high-level goal
goal = "Write a short, suspenseful scene for a children's story about the Apollo 11 moon landing, highlighting the danger."
# Call the execution function from the cell above
execute_and_display(goal, config, client, pc)